In [1]:
#!pip3 install -q -U google-genai

In [2]:
import dotenv
import os
import modules.tools as tools
dotenv.load_dotenv(dotenv_path=".env")

True

In [4]:
registered_tools = [tools.YahooFinance(), tools.FMP(), tools.FinnHub()]
toolsList = [ tool.to_dict() for tool in registered_tools ]
executionMap= { tool.name: tool for tool in registered_tools }

executionMap

{'Yahoo Finance Stock Quote': <modules.tools.YahooFinance at 0x1731c758ec0>,
 'Financial Modeling Prep Quote': <modules.tools.FMP at 0x1730bf5e3c0>,
 'FinnHub News': <modules.tools.FinnHub at 0x1731c9376e0>}

In [5]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

symbol = "AAPL"
report = "all"  # Options: income statement, balance sheet, cash flow, all
benchmark = "SPY"  # S&P 500 ETF as benchmark
systemPrompt = f"""You are a helpful Finance assistant that explains things in a few words.
          You have the following tools:
            {toolsList}
            
            You should only use the tools listed above.
            When you use a tool, you must call the API exactly as shown above.
            You must always include the symbol in the API call.
            You must always include the step in the API call.
            You must always use double quotes for the JSON keys and string values in the API call.
            You must never use single quotes in the API call.
            You must never call any API that is not listed above.
            You must never make up any API calls.
            You must never repeat any steps.
            
            Here is an example of how to use the tools:
            User : What is the company overview for "AAPL".
             To gather the company overview, I will use the "Company Overview" tool.
            
   
            ### TAGs for tool usage:
            - To use a tool, you must wrap the API call in <InvokeTool> and </InvokeTool> tags.
            -  For thinking, you must wrap your thoughts in <Thought> and </Thought> tags.
            -  For final answers, you must wrap your answer in <FinalAnswer> and </FinalAnswer> tags.
            -  If you need users to provide more information, you must wrap your request in <RequestMoreInfo> and </RequestMoreInfo> tags.
            -  If you are going to use a tool, you must always think first. Get the concern of the user query with <NeedApproval>, decide which tool to use, and then use the tool.
           
            ### Thinking Process: 
              Based on the user query, decide which tools to use and in what order.
              After using a tool, analyze the result and decide the next step.
              Continue this process until you have enough information to answer the user's query.
              Finally, provide a comprehensive answer to the user's query.
              
            Conversation:
               
"""
print(systemPrompt)

You are a helpful Finance assistant that explains things in a few words.
          You have the following tools:
            [{'name': 'Yahoo Finance Stock Quote', 'description': 'Get the latest stock quote for a given symbol from Yahoo Finance.', 'api': '{ ""symbol": "AAPL"}'}, {'name': 'Financial Modeling Prep Quote', 'description': 'Get the latest stock quote for a given symbol from Financial Modeling Prep (FMP).', 'api': '{ ""symbol": "AAPL"}'}, {'name': 'FinnHub News', 'description': 'Get the latest financial news for a given symbol from FinnHub.', 'api': '{ ""symbol": "AAPL"}'}]
            
            You should only use the tools listed above.
            When you use a tool, you must call the API exactly as shown above.
            You must always include the symbol in the API call.
            You must always include the step in the API call.
            You must always use double quotes for the JSON keys and string values in the API call.
            You must never use sing

In [6]:
import modules.parser as parser
parser = parser.XmlParser()
# parsedResponse = parser.parse_all(response.text.replace('\n', ' '))
# print(parsedResponse)

In [7]:
import json

In [12]:
prompt = systemPrompt
userText =""
continueFlag = False
exitFlag = ""
debug = False
while userText.lower() != "exit" or exitFlag.lower() != "y":
    if not continueFlag:
        userText = input("User: ")
        if userText.lower() == "exit":
            break
        prompt += "\n User:"+ userText
        continueFlag = False
    if debug:
        print('Prompt: ')
        print(prompt)
    response = client.models.generate_content(
        model="gemini-2.5-flash", contents=prompt
    )
    if debug:
        print('Model response: ')
        print(response.text)
    prompt = prompt + response.text
    actions = parser.parse_all(response.text.replace('\n', ' '))
    if debug:
        print('Actions: ')
        print(actions)
    if not actions:
        continue
    if len(actions) >= 1:
        actions = { action['action']: action for action in actions if action.get("action","") != "Final Answer" }
        if "NeedApproval" in actions:
            actions.pop("NeedApproval")
            if "InvokeTool" in actions:
                #result=input("Need to Call " + actions["InvokeTool"].get("name","") + " Y/y to continue...)")
                result=input("Need to Call " + actions["InvokeTool"]['parameters']['name'] + ". Type Y/y to continue...)")
            else:
                result=input("Need User Approval Y/y to continue...)")
            if result.lower() != "y":
                print("Exiting...")
                break
            actions["NeedApproval"] = {"action":"NeedApproval", "content":"User approved to continue."}
            prompt += "\n User:"+ actions["NeedApproval"].get("content","")
        if 'InvokeTool' in actions:
            action = actions["InvokeTool"]
            tools_name = action["parameters"]["name"]
            tool_params = action["parameters"]["api"]
            result = executionMap[tools_name].invoke(**json.loads(tool_params))
            actions["Tool result"] = {"action":"Tool result", "content":result}
            prompt += "\n User:"+ str(result)
            continueFlag = True
        if "FinalAnswer" in actions:
            content = actions["FinalAnswer"]['parameters']['content']
            print("Final Answer: "+ str(content))
            exitFlag = input("Do you want to exit? Type Y/y to exit...")
            if exitFlag.lower() == "y":
                print('Thanks for chatting! Goodbye!')
                break
            

User:  Summarize the latest news from Apple and give me today's stock price.
Need to Call FinnHub News. Type Y/y to continue...) y


Invoking FinnHub News with arguments {'symbol': 'AAPL', 'step': 1}


Need to Call Yahoo Finance Stock Quote. Type Y/y to continue...) y


Invoking Yahoo Finance Stock Quote with arguments {'symbol': 'AAPL', 'step': 2}
Final Answer: Apple is deeply involved in AI, planning smart glasses and integrating AI into its products. The iPhone 17 is seeing strong demand, though some analysts are cautious about future models' expectations. Apple faces legal scrutiny regarding chip royalties and data collection. The stock price is **$254.04**.


Do you want to exit? Type Y/y to exit... y


Thanks for chatting! Goodbye!
